# Chpater 6

## Fig. 6.3
Two component pathway

In [ ]:
using ModelingToolkit
using Catalyst
using DifferentialEquations
using Plots
Plots.default(linewidth=2)

In [ ]:
rn = @reaction_network begin
    (k1 * L, km1), R <--> RL
    k2, P + RL --> Ps + RL
    k3, Ps--> P
end

In [ ]:
setdefaults!(rn, [
    :R => 3.,
    :RL => 0.,
    :P => 8.0,
    :Ps => 0.,
    :L => 0.,
    :k1 => 5.,
    :km1 => 1.,
    :k2 => 6.,
    :k3 => 2.,
])

osys = convert(ODESystem, rn; remove_conserved = true)

In [ ]:
observed(osys)

In [ ]:
@unpack L = osys
idx = findfirst(isequal(L), parameters(osys))

### Fig. 6.3 A

In [ ]:
cb1 = PresetTimeCallback([1.0], i -> i.p[idx] = 3.)
cb2 = PresetTimeCallback([3.0], i -> i.p[idx] = 0.)
cbs = CallbackSet(cb1, cb2)
prob = ODEProblem(osys, [], (0., 10.))

In [ ]:
sol = solve(prob, callback=cbs)
@unpack RL, Ps = osys

fig = plot(sol, idxs=[RL, Ps], labels= ["RL" "P*"])
plot!(fig, t -> 3 * (1<=t<=3), label="Ligand", line=(:black, :dash), linealpha=0.7)
plot!(fig, title="Fig. 6.3 (A)", xlabel="Time", ylabel="Concentration")

### Fig 6.3 B

In [ ]:
lrange = 0:0.01:1

prob_func = function(prob, i, repeat)
    p = copy(prob.p)
    p[idx] = lrange[i]
    remake(prob, p=p)
end

In [ ]:
prob = SteadyStateProblem(osys, [])

In [ ]:
eprob = EnsembleProblem(prob; prob_func)

In [ ]:
sim = solve(eprob, DynamicSS(Rodas5()); trajectories=length(lrange));

In [ ]:
pstar = map(s->s[Ps], sim)
rl = map(s->s[RL], sim)
plot(lrange, [pstar rl], label=["P*" "RL"], title="Fig. 6.3 (B)",
xlabel="Ligand", ylabel="Steady-state concentration", xlims=(0, 1), ylims=(0, 8))

## Fig 6.5

Model of G-protein signalling pathway

In [ ]:
using ModelingToolkit
using Catalyst
using DifferentialEquations
using Plots
Plots.default(linewidth=2)

In [ ]:
rn = @reaction_network begin
    (kRL * L, kRLm), R <--> RL
    kGa, G + RL --> Ga + Gbg + RL
    kGd0, Ga --> Gd
    kG1, Gd + Gbg --> G
end

In [ ]:
setdefaults!(rn, [
    :kRL => 2e6,
    :kRLm => 0.01,
    :kGa => 1e-5,
    :kGd0 => 0.11,
    :kG1 => 1,
    :R => 4e3,
    :RL => 0.,
    :G => 1e4,
    :Ga => 0.,
    :Gd => 0.,
    :Gbg => 0.,
    :L => 0.
])

osys = convert(ODESystem, rn; remove_conserved = true)

In [ ]:
observed(osys)

In [ ]:
equations(osys)

In [ ]:
@unpack L = osys
idx = findfirst(isequal(L), parameters(osys))

### Fig 6.5 A

In [ ]:
cb1 = PresetTimeCallback([200.0], i -> begin i.p[idx] = 1e-9; set_proposed_dt!(i, 0.01) end)
cb2 = PresetTimeCallback([800.0], i -> begin i.p[idx] = 0.0; set_proposed_dt!(i, 0.01) end)
cbs = CallbackSet(cb1, cb2)
prob = ODEProblem(osys, [], (0., 1200.))

In [ ]:
sol = solve(prob, Rodas5(), callback=cbs, abstol=1e-8, reltol=1e-8, saveat=0:10:1200)
@unpack RL, Ga = osys

fig = plot(sol, idxs=[RL, Ga], title="Fig 6.05 (A)", xlabel="Time", ylabel="Abundance")

### Fig 6.5 B

In [ ]:
lrange = range(0, 20 * 1e-9, 101)

prob_func = function(prob, i, repeat)
    p = copy(prob.p)
    p[idx] = lrange[i]
    remake(prob, p=p)
end

In [ ]:
prob = SteadyStateProblem(osys, [])

In [ ]:
eprob = EnsembleProblem(prob; prob_func)

In [ ]:
sim = solve(eprob, DynamicSS(Rodas5()); trajectories=length(lrange));

In [ ]:
ga = map(s->s[Ga], sim)
rl = map(s->s[RL], sim)
plot(lrange .* 1e9, [ga rl], label=["Ga" "RL"], title="Fig. 6.5 (B)",
xlabel="Ligand (nM)", ylabel="Steady-state abundance", xlims=(0, 20), ylims=(0, 3500))

## Fig 6.07

Goldbeter Koshland switch

In [ ]:
using Plots
Plots.default(linewidth=2)

f607(w, K1, K2) = w * (1 - w + K1)/((1 - w) * (w + K2))

plot(title = "Fig 6.07", xlims=(0, 3), ylims=(0, 1))
plot!(x -> f607(x, 1, 1), y -> y, 0, 1, label="K1=K2=1")
plot!(x -> f607(x, 0.1, 0.1), y -> y, 0, 1, label="K1=K2=0.1")
plot!(x -> f607(x, 0.01, 0.01), y -> y, 0, 1, label="K1=K2=0.01")

## Fig 6.14

Model of E. coli chemotaxis signalling pathway

In [ ]:
using ModelingToolkit
using Catalyst
using DifferentialEquations
using Plots
Plots.default(linewidth=2)

In [ ]:
rn = @reaction_network begin
    mm(Am, k1 * BP, KM1), Am ⇒ A
    mm(AmL, k2 * BP, KM2), AmL ⇒ AL
    km1 * R , A ⇒ Am
    km2 * R , AL ⇒ AmL
    (k3 * L, km3), Am <--> AmL
    (k4 * L, km4), A <--> AL
    (k5 * Am, km5), B <--> BP
end

In [ ]:
setdefaults!(rn, [
    :Am => 0.0360,
    :AmL => 1.5593,
    :A => 0.0595,
    :AL => 0.3504,
    :B => 0.7356,
    :BP => 0.2644,
    :k1 => 200,
    :k2 => 1,
    :k3 => 1,
    :km1 => 1,
    :km2 => 1,
    :km3 => 1,
    :k5 => 0.05,
    :km5 => 0.005,
    :k4 => 1,
    :km4 => 1,
    :KM1 => 1,
    :KM2 => 1,
    :R => 1,
    :L => 20
])

In [ ]:
osys = convert(ODESystem, rn; remove_conserved = true)

In [ ]:
observed(osys)

In [ ]:
equations(osys)

In [ ]:
@unpack L = osys
idx = findfirst(isequal(L), parameters(osys))

In [ ]:
cb1 = PresetTimeCallback([10.0], i -> begin i.p[idx] = 40; set_proposed_dt!(i, 0.01) end)
cb2 = PresetTimeCallback([30.0], i -> begin i.p[idx] = 80; set_proposed_dt!(i, 0.01) end)
cbs = CallbackSet(cb1, cb2)
prob = ODEProblem(osys, [], (0., 50.))

In [ ]:
sol = solve(prob, callback=cbs)

@unpack Am = osys

plot(sol, idxs=[Am], title="Fig 6.14", xlabel="Time", ylabel="Active CheA ([Am])", ylims=(0.01, 0.04), legend=false)

## Fig 6.16

Model of apoptosis signalling pathway

In [ ]:
using Catalyst
using ModelingToolkit
using DifferentialEquations
using Plots
Plots.default(linewidth=2)

In [ ]:
rn = @reaction_network begin
    (k1, k2), 0 <--> C8
    k3 * (C3s + I), C8 --> C8s
    k4, C8s --> 0
    (k5, k6), C8s + BAR <--> C8sBAR
    (k7, k8), 0 <--> C3
    k9 * C8s, C3 --> C3s
    k10, C3s --> 0
    (k11, k12), C3s + IAP <--> C3sIAP
    (k13, k14), 0 <--> BAR
    (k15, k16 + k17 * C3s), 0 <--> IAP
    k18, C8sBAR --> 0
    k19, C3sIAP --> 0
end

In [ ]:
setdefaults!(rn, [
    :k1 => 507,
    :k2 => 3.9e-3,
    :k3 => 1e-5,
    :k4 => 5.8e-3,
    :k5 => 5e-4,
    :k6 => 0.21,
    :k7 => 81.9,
    :k8 => 3.9e-3,
    :k9 => 5.8e-6,
    :k10 => 5.8e-3,
    :k11 => 5e-4,
    :k12 => 0.21,
    :k13 => 40,
    :k14 => 1e-3,
    :k15 => 464,
    :k16 => 1.16e-2,
    :k17 => 3e-4,
    :k18 => 1.16e-2,
    :k19 => 1.73e-2,
    :I => 0.0,
    :C8 => 1.3E5,
    :C8s => 0.0,
    :C3 => 0.21E5,
    :C3s => 0.0,
    :BAR => 0.4E5,
    :IAP => 0.4E5,
    :C8sBAR => 0.0,
    :C3sIAP => 0.0
])

osys = convert(ODESystem, rn; remove_conserved = true)
equations(osys)

In [ ]:
@unpack I = osys
idx = findfirst(isequal(I), parameters(osys))

In [ ]:
cb1 = PresetTimeCallback([100.0], i -> begin i.p[idx] = 200; set_proposed_dt!(i, 0.01) end)
cb2 = PresetTimeCallback([1200.0], i -> begin i.p[idx] = 0; set_proposed_dt!(i, 0.01) end)
cbs = CallbackSet(cb1, cb2)
prob = ODEProblem(osys, [], (0., 1800.))

In [ ]:
sol = solve(prob, callback=cbs)

@unpack C8s, C3s = osys

plot(sol, idxs=[C8s, C3s], title="Fig 6.16", xlabel="Time", ylabel="Concentration", legend=:right, rightmargin=5*Plots.mm)

## Fig 6.18

Model of calcium-induced calcium release in hepatocytes

In [ ]:
using Catalyst
using ModelingToolkit
using DifferentialEquations
using Plots
Plots.default(linewidth=2)

In [ ]:
rn = @reaction_network begin
    (k1 * I, km1), R <--> RI
    (k2 * C, km2), RI <--> RIC
    (k3 * C, km3), RIC <--> RICC
    vr * (γ0 + γ1 * RIC) * (Cer - C), 0 --> C
    hill(C, p1, p2, 4), C ⇒ 0
end

In [ ]:
setdefaults!(rn, [
    :γ0 => 0.1,
    :γ1 => 20.5,
    :p1 => 8.5,
    :p2 => 0.065,
    :k1 => 12,
    :k2 => 15,
    :k3 => 1.8,
    :km1 => 8,
    :km2 => 1.65,
    :km3 => 0.21,
    :Cer => 8.37,
    :vr => 0.185,
    :I => 0,
    :C => 0,
    :R => 1,
    :RI => 0,
    :RIC => 0,
    :RICC => 0
])

osys = convert(ODESystem, rn; remove_conserved = true)
equations(osys)

In [ ]:
observed(osys)

### Fig 6.18 (A)

In [ ]:
@unpack I = osys
prob = ODEProblem(osys, [], (0., 25.), [I => 2.0])

In [ ]:
sol = solve(prob)

In [ ]:
@unpack C, RIC, RICC = osys
plot(sol, idxs=[C, RIC, RICC], title="Fig 6.18 (A)", xlabel="Time", ylabel="Abundance", legend=:topright)

### Fig 6.18 (B)

In [ ]:
idx = findfirst(isequal(I), parameters(osys))
cb1 = PresetTimeCallback([20.0], i -> begin i.p[idx] = 0.7; set_proposed_dt!(i, 0.01) end)
cb2 = PresetTimeCallback([60.0], i -> begin i.p[idx] = 1.2; set_proposed_dt!(i, 0.01) end)
cb3 = PresetTimeCallback([90.0], i -> begin i.p[idx] = 4.0; set_proposed_dt!(i, 0.01) end)
cbs = CallbackSet(cb1, cb2, cb3)
prob = ODEProblem(osys, [], (0., 120.))

In [ ]:
sol = solve(prob, callback=cbs)
@unpack C = osys
plot(sol, idxs=[C], title="Fig 6.18 (B)", xlabel="Time", ylabel="Ca concentration", legend=false, ylim=(0, 2.5))

## Runtime information

In [ ]:
import InteractiveUtils
InteractiveUtils.versioninfo()

In [ ]:
import Pkg
Pkg.status()